In [7]:
# Step 1. Generate toy aligned data
import pandas as pd
import numpy as np

dates = pd.date_range("2023-01-01", periods=100)
tickers = [f"T{i}" for i in range(10)]
index = pd.MultiIndex.from_product([dates, tickers], names=["date", "ticker"])

df = pd.DataFrame(index=index)
np.random.seed(42)

df["realized"] = np.random.normal(0.2, 0.05, size=len(df))
df["implied"] = np.random.normal(0.18, 0.05, size=len(df))
df["forecast"] = df["realized"] + np.random.normal(0, 0.02, size=len(df))
df["signal"] = df["forecast"] - df["implied"]

# Step 2. Rank and meta-label
df["signal_rank"] = df.groupby("date")["signal"].rank(pct=True)
df["implied_rank"] = df.groupby("date")["implied"].rank(pct=True)
df["realized_rank"] = df.groupby("date")["realized"].rank(pct=True)

threshold = 0.1
df["target"] = ((df["realized_rank"] > df["implied_rank"] + threshold) & (df["realized"] > 0.1)).astype(int)

# Step 3. Simulate trade decisions from target or ML preds
df["position"] = df["target"]  # stand-in for now

# Step 4. Toy PnL proxy
df["proxy_return"] = df["realized"] - df["implied"]
df["trade_pnl"] = df["position"].shift(1) * df["proxy_return"]  # lag trade decision

# Optional slippage
df["slippage"] = df["position"].diff().abs().fillna(0) * 0.002
df["net_pnl"] = df["trade_pnl"] - df["slippage"]

In [11]:
price_matrix = pd.DataFrame(index=dates, columns=tickers)
#vol_matrix_df = pd.DataFrame(index=dates, columns=tickers)
#implied_vol_matrix_df = pd.DataFrame(index=dates, columns=tickers)
log_returns = np.random.normal(0, 1, size=(len(dates), len(tickers)))
price_matrix[:] = np.exp(np.cumsum(log_returns, axis=0))
stock_vols=np.random.uniform(0.15,0.45,size=len(tickers))
stock_vols_matrix=np.tile(stock_vols, (len(dates), 1))
log_returns=log_returns*stock_vols_matrix/np.sqrt(252)
initial_price=np.random.uniform(10,100,size=len(tickers))
initial_price_matrix=np.tile(initial_price, (len(dates), 1))
price_matrix[:] = initial_price_matrix*np.exp(np.cumsum(log_returns, axis=0))
price_matrix


realized_vols=np.random.normal(0, 1, size=(len(dates), len(tickers)))
realized_vols_matrix=np.exp(np.cumsum(realized_vols, axis=0))
realized_vols_matrix_df=pd.DataFrame(realized_vols_matrix,index=dates,columns=tickers)
realized_vols_matrix_df = compute_realized_vol(price_matrix, window=5)


implied_vols=np.random.normal(0, 1, size=(len(dates), len(tickers)))
implied_vols_matrix=np.exp(np.cumsum(implied_vols, axis=0))
implied_vols_matrix_df=pd.DataFrame(implied_vols_matrix,index=dates,columns=tickers)


implied_vols_matrix_df = simulate_ou_vol_matrix(dates, tickers, mean=0.25, vol=0.05, seed=1)





In [12]:
trades=pd.DataFrame(index=dates,columns=tickers)
trades[:]=np.random.randint(0,3,size=(len(dates),len(tickers)))-1
trades





,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
2023-01-01,-1,1,1,0,-1,0,1,0,-1,1
2023-01-02,0,1,-1,0,0,1,-1,0,1,0
2023-01-03,0,0,-1,0,1,0,-1,1,-1,0
2023-01-04,-1,1,0,-1,1,1,1,0,1,1
2023-01-05,0,0,-1,0,-1,1,-1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2023-04-06,1,0,-1,-1,0,1,-1,0,1,-1
2023-04-07,1,1,0,1,1,1,0,-1,1,-1
2023-04-08,1,1,1,-1,-1,1,1,0,-1,-1
2023-04-09,0,0,-1,1,1,0,1,0,0,1


In [13]:
entry_prices=implied_vols_matrix_df.copy()
entry_prices
exit_prices=realized_vols_matrix_df.copy().shift(-5)
exit_prices

pnl=trades*(exit_prices-entry_prices)
pnl










,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
2023-01-01,0.005032,0.30163,0.245876,0.0,-0.084279,-0.0,0.13448,0.0,0.110325,0.030453
2023-01-02,-0.0,0.229258,-0.217096,0.0,0.0,-0.04109,-0.108913,0.0,-0.137256,-0.0
2023-01-03,-0.0,0.0,-0.071947,-0.0,-0.041534,-0.0,-0.156784,-0.003385,0.018188,0.0
2023-01-04,0.082632,0.214176,-0.0,-0.110109,-0.080996,-0.008057,0.204779,0.0,-0.021925,0.196966
2023-01-05,-0.0,0.0,0.042571,0.0,0.001239,-0.023781,-0.200902,0.0,-0.0,0.199958
...,...,...,...,...,...,...,...,...,...,...
2023-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def compute_vol_pnl(trades, implied_vols, realized_vols, holding_period=5, slippage=0.0):
    """
    Computes PnL from trading implied vs. realized vol.
    Assumes trades are made at t using implied_vols[t] and closed at t+H using realized_vols[t+H].

    Parameters:
        trades         : DataFrame[date, ticker] with -1, 0, 1 signals
        implied_vols   : DataFrame[date, ticker] with implied vols
        realized_vols  : DataFrame[date, ticker] with realized vols
        holding_period : int, number of days to hold the position
        slippage       : float, slippage cost per trade

    Returns:
        net_pnl        : DataFrame[date, ticker] with PnL
    """
    entry_vol = implied_vols.astype(float)
    exit_vol = realized_vols.shift(-holding_period).astype(float)
    
    raw_pnl = trades * (exit_vol - entry_vol)
    turnover = trades.diff().abs()
    cost = turnover * slippage
    net_pnl = raw_pnl - cost

    return net_pnl.iloc[:-holding_period]  # drop rows where exit_vol is NaN

In [18]:
realized_vols_matrix_df.head(6)

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06,0.092415,0.145599,0.376201,0.284859,0.154154,0.482248,0.164395,0.407806,0.343288,0.32792


In [15]:
compute_vol_pnl(trades, implied_vols_matrix_df, realized_vols_matrix_df, holding_period=5, slippage=0.0)

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,-0.0,-0.080571,-0.133396,-0.0,-0.0,0.250539,-0.197386,0.0,0.056618,0.0
2023-01-03,-0.0,-0.0,0.011498,-0.0,-0.103334,0.0,-0.34986,0.159343,-0.150528,0.0
2023-01-04,0.079703,-0.069801,0.0,-0.035188,-0.078752,0.240082,0.372311,0.0,0.047232,-0.011015
2023-01-05,-0.0,-0.0,-0.14632,-0.0,0.040406,0.06159,-0.381102,0.0,0.0,0.023948
...,...,...,...,...,...,...,...,...,...,...
2023-04-01,-0.027189,-0.112862,-0.0,0.0,-0.0,-0.215921,0.0,0.050398,-0.113082,-0.034173
2023-04-02,-0.02398,0.105811,-0.113563,-0.088033,-0.024487,0.277987,0.002749,-0.097825,0.041611,0.0
2023-04-03,0.024679,-0.095076,0.112464,-0.032035,0.014195,-0.188571,0.0,-0.024867,-0.0,0.0
2023-04-04,-0.037371,-0.0,-0.076828,0.012225,0.00392,0.174641,0.042325,-0.042996,0.0,0.120266


In [10]:
def simulate_ou_vol_matrix(dates, tickers, mean=0.2, vol=0.05, theta=0.1, dt=1/252, seed=None):
    """
    Simulate a mean-reverting log-vol process across tickers and dates.
    Returns a DataFrame (dates × tickers).
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_dates, n_tickers = len(dates), len(tickers)
    log_vols = np.zeros((n_dates, n_tickers))
    log_vols[0, :] = np.log(mean)

    for t in range(1, n_dates):
        dW = np.random.normal(0, 1, size=n_tickers)
        log_vols[t] = (
            log_vols[t - 1]
            + theta * (np.log(mean) - log_vols[t - 1]) * dt
            + vol * np.sqrt(dt) * dW
        )

    vols = np.exp(log_vols)
    return pd.DataFrame(vols, index=dates, columns=tickers)

    
def compute_realized_vol(price_df, window=5):
    price_df = price_df.astype(float)  # ensure clean dtype
    log_returns = np.log(price_df / price_df.shift(1))
    realized_vol = log_returns.rolling(window).std() * np.sqrt(252)  # annualized
    return realized_vol

In [40]:
import py_vollib.black_scholes
import py_vollib_vectorized
import pandas as pd
import numpy as np


def compute_straddle_price(spot, strike, ttm, r, q, sigma):
    #print("spot", spot)
    #print("strike", strike)
    #print("ttm", ttm)
    #print("r", r)
    #print("q", q)
    #print("sigma", sigma)
    call_prices = py_vollib.black_scholes.black_scholes('c', spot.values.flatten(),strike.values.flatten() , ttm, r, sigma.values.flatten(), return_as='array')  
    put_prices = py_vollib.black_scholes.black_scholes('p', spot.values.flatten(),strike.values.flatten() , ttm, r, sigma.values.flatten(), return_as='array')  
    #option_price = py_vollib.black_scholes('c', spot,strike, ttm, r, sigma, return_as='array')  
    return call_prices+put_prices



holding_period=5.0
r=0
q=0

straddle_prices=compute_straddle_price(price_matrix, price_matrix, holding_period/252, r, q, implied_vols_matrix_df)

straddle_prices.reshape(price_matrix.shape) 





array([[1.10528326, 2.73641788, 1.89067055, 0.40962272, 1.11138817,
        1.87027622, 0.42275576, 0.33547314, 1.41398217, 1.08540442],
       [1.14282135, 2.65633071, 1.8505887 , 0.40435135, 1.1527652 ,
        1.85724754, 0.41139634, 0.3447403 , 1.43486667, 1.09679588],
       [1.1583454 , 2.78523721, 1.92772053, 0.39787734, 1.20729298,
        1.86347746, 0.41664219, 0.33030949, 1.43049592, 1.08034712],
       [1.16626632, 2.86349117, 1.85790211, 0.39831733, 1.24692224,
        1.85331793, 0.4245986 , 0.33208301, 1.43844151, 1.07604862],
       [1.14627327, 2.78865329, 1.87924354, 0.39742706, 1.23976641,
        1.83934425, 0.41970089, 0.34065514, 1.43695192, 1.04882634],
       [1.15620936, 2.76808528, 1.87595587, 0.41376563, 1.23929199,
        1.8451261 , 0.40704572, 0.34114108, 1.45395595, 1.01733419],
       [1.16019193, 2.77130368, 1.87069977, 0.42273777, 1.24939913,
        1.8977522 , 0.39752472, 0.34324395, 1.45755919, 0.99768041],
       [1.1712664 , 2.93167589, 1.8731199

In [35]:
py_vollib.black_scholes.black_scholes('c', price_matrix.values.flatten(),price_matrix.values.flatten() , 20/252, 0, .2, return_as='array')  











    

array([0.88415535, 2.18895787, 1.51241453, 0.32767177, 0.88903888,
       1.49610039, 0.33817735, 0.26835688, 1.13109457, 0.86825355,
       0.90951867, 2.12899118, 1.48281613, 0.32454987, 0.91962798,
       1.49648623, 0.32728713, 0.27643191, 1.14664812, 0.87805531,
       0.91764019, 2.24683588, 1.54618754, 0.31973963, 0.95969612,
       1.50671188, 0.33164125, 0.26559352, 1.1430038 , 0.86330071,
       0.9271268 , 2.30164242, 1.48596064, 0.31958646, 0.98839234,
       1.50172126, 0.33810593, 0.2678071 , 1.15032315, 0.85843112,
       0.91322263, 2.24428684, 1.50628606, 0.31972171, 0.9848034 ,
       1.4904504 , 0.33538459, 0.27451644, 1.14314082, 0.83476201,
       0.92169677, 2.2339648 , 1.50719205, 0.33109549, 0.98427177,
       1.49813033, 0.32507642, 0.27309485, 1.15623297, 0.80812664,
       0.92399892, 2.23903688, 1.50838433, 0.33864733, 0.99295494,
       1.53800632, 0.31663518, 0.27397362, 1.15805904, 0.79031054,
       0.93503938, 2.35927055, 1.50789312, 0.34238664, 0.99516